In [ ]:
"""
Centralized training (CSV + 预处理版)，参考原 centralized notebook 流程。
直接在下方 args 配置数据路径/参数，运行文件或在 Notebook 中 `%run` 即可。
"""

from __future__ import annotations

from argparse import Namespace
from pathlib import Path
import os
import sys
from typing import Optional, List

import numpy as np
import torch
from torch.utils.data import DataLoader

parent = Path(os.path.abspath("")).resolve().parents[0]
if str(parent) not in sys.path:
    sys.path.insert(0, str(parent))

from ml.models.mlp import MLP
from ml.models.rnn import RNN
from ml.models.lstm import LSTM
from ml.models.gru import GRU
from ml.models.cnn import CNN
from ml.models.rnn_autoencoder import DualAttentionAutoEncoder
from ml.utils.data_utils import (
    read_data,
    handle_nans,
    to_train_val,
    to_Xy,
    generate_time_lags,
    scale_features,
    remove_identifiers,
    to_timeseries_rep,
    to_torch_dataset,
)
from ml.utils.train_utils import train, test



args = Namespace(
    data_path="../dataset/turbine_prediction_selected_features_400turbines.csv",  # 集中训练的 CSV
    test_size=0.2,
    targets=["power_output"],
    num_lags=24,
    identifier="GSRN",
    nan_constant=0,
    x_scaler="minmax",
    y_scaler="minmax",
    criterion="mse",
    epochs=20,          # 400 台时适当降低
    lr=0.001,
    optimizer="adam",
    batch_size=512,     # 尽量大以减少步数
    max_grad_norm=0.0,
    reg1=0.0,
    reg2=0.0,
    device="cuda" if torch.cuda.is_available() else "mps",
    seed=0,
    forecast_horizon=3,
    model_name="lstm",  # mlp/rnn/lstm/gru/cnn/da_encoder_decoder
    patience=5,
    num_workers=8,
    pin_memory=False,
    use_carbontracker=False,
)
# -----------------------------------


def make_preprocessing():
    df = read_data(args.data_path)
    df = handle_nans(train_data=df, constant=args.nan_constant, identifier=args.identifier)
    train_size = 1 - args.test_size
    train_data, val_data = to_train_val(df, train_size=train_size)

    X_train, X_val, y_train, y_val = to_Xy(
        train_data=train_data, val_data=val_data, targets=args.targets, ignore_cols=[]
    )

    # scale
    X_train, X_val, x_scalers = scale_features(
        train_data=X_train, val_data=X_val, scaler=args.x_scaler, per_area=False, identifier=args.identifier
    )
    y_train, y_val, y_scalers = scale_features(
        train_data=y_train, val_data=y_val, scaler=args.y_scaler, per_area=False, identifier=args.identifier
    )

    # lags
    X_train = generate_time_lags(X_train, args.num_lags, identifier=args.identifier)
    X_val = generate_time_lags(X_val, args.num_lags, identifier=args.identifier)
    y_train = generate_time_lags(y_train, args.num_lags, identifier=args.identifier, is_y=True)
    y_val = generate_time_lags(y_val, args.num_lags, identifier=args.identifier, is_y=True)

    # remove identifiers
    X_train, y_train, X_val, y_val = remove_identifiers(X_train, y_train, X_val, y_val)
    num_features = len(X_train.columns) // args.num_lags

    # to np (timeseries rep)
    X_train = to_timeseries_rep(X_train.to_numpy(), num_lags=args.num_lags, num_features=num_features)
    X_val = to_timeseries_rep(X_val.to_numpy(), num_lags=args.num_lags, num_features=num_features)
    y_train, y_val = y_train.to_numpy(), y_val.to_numpy()

    return X_train, X_val, y_train, y_val, num_features, x_scalers, y_scalers


def get_model_local(
    model: str,
    input_dim: int,
    out_dim: int,
    lags: int = 10,
    exogenous_dim: int = 0,
    seed: int = 0,
    **kwargs,
):
    torch.manual_seed(seed)
    if model == "mlp":
        return MLP(input_dim=input_dim, layer_units=[256, 128, 64], num_outputs=out_dim)
    if model == "rnn":
        return RNN(
            input_dim=input_dim,
            rnn_hidden_size=128,
            num_rnn_layers=1,
            rnn_dropout=0.0,
            layer_units=[128],
            num_outputs=out_dim,
            matrix_rep=True,
            exogenous_dim=exogenous_dim,
        )
    if model == "lstm":
        return LSTM(
            input_dim=input_dim,
            lstm_hidden_size=128,
            num_lstm_layers=1,
            lstm_dropout=0.0,
            layer_units=[128],
            num_outputs=out_dim,
            matrix_rep=True,
            exogenous_dim=exogenous_dim,
        )
    if model == "gru":
        return GRU(
            input_dim=input_dim,
            gru_hidden_size=128,
            num_gru_layers=1,
            gru_dropout=0.0,
            layer_units=[128],
            num_outputs=out_dim,
            matrix_rep=True,
            exogenous_dim=exogenous_dim,
        )
    if model == "cnn":
        return CNN(num_features=input_dim, lags=lags, exogenous_dim=exogenous_dim, out_dim=out_dim)
    if model == "da_encoder_decoder":
        return DualAttentionAutoEncoder(input_dim=input_dim, architecture="lstm", matrix_rep=True)
    raise NotImplementedError("指定模型未实现，可选：mlp/rnn/lstm/gru/cnn/da_encoder_decoder")


def main():
    torch.manual_seed(args.seed)
    X_train, X_val, y_train, y_val, num_features, x_scalers, y_scalers = make_preprocessing()

    # dataloaders
    train_loader = to_torch_dataset(
        X_train,
        y_train,
        num_lags=args.num_lags,
        num_features=num_features,
        batch_size=args.batch_size,
        shuffle=True,
    )
    val_loader = to_torch_dataset(
        X_val,
        y_val,
        num_lags=args.num_lags,
        num_features=num_features,
        batch_size=args.batch_size,
        shuffle=False,
    )

    # model
    input_dim = num_features if args.model_name != "mlp" else num_features * args.num_lags
    model = get_model_local(
        model=args.model_name,
        input_dim=input_dim,
        out_dim=y_train.shape[1],
        lags=args.num_lags,
        exogenous_dim=0,
        seed=args.seed,
    )

    device = torch.device(args.device)
    best_model = train(
        model=model,
        train_loader=train_loader,
        test_loader=val_loader,
        device=device,
        epochs=args.epochs,
        optimizer=args.optimizer,
        lr=args.lr,
        early_stopping=True,
        patience=args.patience,
        use_carbontracker=args.use_carbontracker,
        log_per=1,
        plot_history=True,
    )

    # final eval
    val_mse, val_rmse, val_mae, val_r2, val_nrmse, _ = test(best_model, val_loader, None, device=device)
    print(f"[Final Val] mse={val_mse:.4f}, rmse={val_rmse:.4f}, mae={val_mae:.4f}, r2={val_r2:.4f}, nrmse={val_nrmse:.4f}")


if __name__ == "__main__":
    main()


/Users/libowen/Library/Python/3.10/lib/python/site-packages/sklearn/impute/_base.py:590: FutureWarning: Currently, when `keep_empty_feature=False` and `strategy="constant"`, empty features are not dropped. This behaviour will change in version 1.8. Set `keep_empty_feature=True` to preserve this behaviour.
  warnings.warn(
INFO logger 2026-01-03 23:12:37,550 | data_utils.py:393 | Observations info in 571313174116253585
INFO logger 2026-01-03 23:12:37,550 | data_utils.py:394 | 	Total number of samples:  8760
INFO logger 2026-01-03 23:12:37,550 | data_utils.py:395 | 	Number of samples for training: 7008
INFO logger 2026-01-03 23:12:37,551 | data_utils.py:396 | 	Number of samples for validation:  1752
INFO logger 2026-01-03 23:12:37,553 | data_utils.py:393 | Observations info in 570714700000106958
INFO logger 2026-01-03 23:12:37,554 | data_utils.py:394 | 	Total number of samples:  8760
INFO logger 2026-01-03 23:12:37,554 | data_utils.py:395 | 	Number of samples for training: 7008
INFO logg